In [160]:
# Import packages

import math
import json
import re
import bs4
import csv
import requests

In [139]:
# Scrape data from Albuquerque Next Request portal which has different url/data format than others

cities = ['Albuquerque']

for city in cities:
    fn = '{}.csv'.format(city)
    with open(fn, 'w') as csvfile:
        fieldnames = ['id', 'status', 'requester', 'request', 'departments', 'cost', 'PoC']
        writer = csv.DictWriter(csvfile, fieldnames = fieldnames, delimiter = ',')

        writer.writeheader()
        
        req = requests.get('https://nextrequest.cabq.gov/requests')

        for i in range(2, 479):
            html = req.text.encode('utf8')
            soup = bs4.BeautifulSoup(html, "html5lib")
            rl = soup.tbody.find_all('tr', class_ = " demo-data-false") 
            for prr in rl:
                cl = list(prr.children)
                writer.writerow({'id': cl[1].a.strong.contents[0],
                                'status': cl[3]['class'][1],
                                'requester': cl[5].get_text().lstrip().rstrip(),
                                'request': cl[7].a.contents[0].lstrip().rstrip(),
                                'departments': cl[9].get_text().lstrip().rstrip(),
                                'cost': cl[11].get_text().lstrip().rstrip().strip('$'),
                                'PoC': cl[13].get_text().lstrip().rstrip()})
            url = 'https://nextrequest.cabq.gov/requests?requests_smart_listing[page]={}'.format(i)
            req = requests.get(url)

In [177]:
# Scrape data from Next Request portals from list of cities given below

cities = ['bainbridgewa', 'cityoflascruces', 'mercerisland', 'miami', 'middleboroughma', 
          'nola', 'oaklandca', 'providenceri', 'sanfrancisco', 'vallejo', 'westsacramento' ]

for city in cities:
    print(city)
    fn = '{}.csv'.format(city)
    with open(fn, 'w') as csvfile:
        fieldnames = ['id', 'status', 'requester', 'request', 'departments', 'PoC']
        writer = csv.DictWriter(csvfile, fieldnames = fieldnames, delimiter = ',')

        writer.writeheader()
        url = 'https://{}.nextrequest.com/requests'.format(city)
        print(url)
        
        req = requests.get(url)
        html = req.text.encode('utf8')
        soup = bs4.BeautifulSoup(html, "html5lib")
        
        num_rec = int(soup.find_all('h2')[1].span.get_text().lstrip().rstrip())
        print(num_rec)
        num_page = math.ceil(num_rec/25)
        print(num_page)

        for i in range(2, num_page + 2):
            rl = soup.tbody.find_all('tr', class_ = " demo-data-false") 
            for prr in rl:
                cl = list(prr.children)
                writer.writerow({'id': cl[1].a.strong.contents[0],
                                'status': cl[3]['class'][1],
                                'requester': cl[5].get_text().lstrip().rstrip(),
                                'request': cl[7].a.contents[0].lstrip().rstrip(),
                                'departments': cl[9].get_text().lstrip().rstrip(),
                                'PoC': cl[11].get_text().lstrip().rstrip()})
            url_2 = url+'?requests_smart_listing[page]={}'.format(i)
            req = requests.get(url_2)
            html = req.text.encode('utf8')
            soup = bs4.BeautifulSoup(html, "html5lib")

oaklandca
https://oaklandca.nextrequest.com/requests
27122
1085
providenceri
https://providenceri.nextrequest.com/requests
2192
88
sanfrancisco
https://sanfrancisco.nextrequest.com/requests
1525
61
vallejo
https://vallejo.nextrequest.com/requests
340
14
westsacramento
https://westsacramento.nextrequest.com/requests
886
36
